In [2]:
import os, glob, platform
import numpy as np
import numpy.matlib
import pickle
import pandas as pd
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne
mne.__version__
from mne.viz import plot_alignment, snapshot_brain_montage
import shutil
from mne.datasets import eegbci
from sklearn.model_selection import train_test_split

# from mne_bids import write_raw_bids, BIDSPath, print_dir_tree, make_dataset_description
# from mne_bids.stats import count_events
import sys


In [3]:
''' VARIABLES '''

dataset = 'eeg'

amount_of_subjects = 1 # Change the range so the process is faster

numC = 8

angles = [i * 180./numC for i in range(numC)]

x_labels = np.array(angles)

resample = False # speeds up the procees but showing worse results overall
resample_frequency = 20 # in Hz, original freq is 500Hz

In [4]:
from toolbox.methods import read_data
all_epochs, all_rawdata = read_data (task = 'main', resample=resample, amount_of_subjects=amount_of_subjects, resample_frequency=resample_frequency)

In [5]:
print(len(all_rawdata))
all_rawdata[0]['metadata'].columns

1


Index(['index', 'subj', 'nblock', 'ntrial', 'nrep', 'trial_type', 'cond-1',
       'cond', 'rDV', 'DV', 'resp', 'deci-2', 'deci-1', 'deci', 'corr-1',
       'r_map', 'correct', 'confi', 'RT', 'd1', 'conf_lvl', 'correct-1', 'd2',
       'd3', 'd4', 'd5', 'd6', 'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'confi-1',
       'conf_lvl-1'],
      dtype='object')

In [77]:
def diagonality(x):
    if x >= 0 and x <= 22.5:
        return 0
    elif x > 22.5 and x <= 67.5:
        return 1
    elif x > 67.5 and x <= 112.5:
        return 0
    elif x > 112.5 and x <= 157.5:
        return 1
    elif x > 157.5 and x <= 180:
        return 0

In [6]:
all_rawdata[0]['metadata'][['o1', 'o2', 'o3', 'o4', 'o5', 'o6']] *= 180 / np.pi
all_rawdata[0]['metadata']['o1'] = all_rawdata[0]['metadata']['o1'].apply(diagonality)

NameError: name 'diagonality' is not defined

In [ ]:
all_rawdata[0]['metadata']['o1']

883     1
896     1
909     1
922     0
935     0
       ..
4240    0
4253    0
4266    0
4279    1
4305    1
Name: o1, Length: 250, dtype: int64

In [96]:
'''7min for all subjects'''
use_timepoint = False
reshape = False
timepoint = 10
X = all_rawdata[0]['epoch_dat']
print(X.shape)
# X = X.reshape(X.shape[0],-1)
y = all_rawdata[0]['metadata']['deci']
y = np.array(y)
print(y.shape)

for i in range(1, amount_of_subjects):

    X_temp = all_rawdata[i]['epoch_dat']
    # X_temp = X_temp.reshape((X_temp.shape[0]*X_temp.shape[1], X_temp.shape[2]))
    # shapes[i+1] = shapes[i] + X_temp.shape[0]
    y_temp = all_rawdata[i]['metadata']['deci']
    # y_temp = np.tile(y_temp, numC)
    # X = np.vstack((X, X_temp))
    X = np.concatenate((X, X_temp))
    y = np.concatenate((y, y_temp))

print(X.shape, y.shape)
if use_timepoint: 
    X = X[:,:,timepoint]
else:
    if reshape:
        X = X.reshape(X.shape[0], -1)
print(X.shape, y.shape)
X = np.einsum('ijk->ikj',X)
X_eye = X[:,:,25]
X = np.delete(X, 25, axis=2)
X = X[:,750:1750,:]
print(X.shape, y.shape)


(250, 32, 2876)
(250,)
(250, 32, 2876) (250,)
(250, 32, 2876) (250,)
(250, 1000, 31) (250,)


In [55]:
var_X = X[:,950:1200,:]

In [19]:
X_1 = X[y==1]
X_1.shape

(169, 2876)

In [50]:
arima_X = X[3,950:1200,25]

In [51]:
arima_X.shape

(250,)

In [52]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
# Load your time series data into a pandas DataFrame
# data = pd.read_csv('your_data.csv')

# Find the best orders automatically
model = auto_arima(arima_X, trace=True, error_action='ignore', suppress_warnings=True)

best_order = model.order
print("Best Order:", best_order)

# Fit the ARIMA model with the best orders to the data
fitted_model = ARIMA(arima_X, order=best_order).fit()

# Analyze the results of the fitted ARIMA model
print(fitted_model.summary())

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-5235.431, Time=0.36 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-5245.638, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-5203.474, Time=0.09 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-5353.844, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-5395.226, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-5362.948, Time=0.11 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.748 seconds
Best Order: (0, 1, 0)
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  250
Model:                 ARIMA(0, 1, 0)   Log Likelihood                2698.613
Date:                Mon, 12 Jun 2023   AIC                          -5395.226
Time:                        13:25:47   BIC                          -5391.708
Sample:                             0   HQIC                 

/Users/junjieji/opt/anaconda3/envs/mne/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [53]:
fitted_model.params

array([1.93687913e-11])

In [119]:
# Define the maximum order to consider
max_order = 5

# Initialize a list to store the best VAR models for each series
best_models = []

# Iterate over each series
for i in range(X.shape[0]):
    series = X[i]
    best_order = None
    best_model = None
    min_residual_sum_squared = float('inf')
    
    # Iterate over different orders
    for p in range(1, max_order + 1):
        # Fit the VAR model with order p
        model = VAR(series)
        results = model.fit(p)
        
        # Calculate the residual sum of squares for the current order
        residual_sum_squared = np.sum(results.resid**2)
        
        # Check if the current order has the minimum residual sum of squares
        if residual_sum_squared < min_residual_sum_squared:
            min_residual_sum_squared = residual_sum_squared
            best_order = p
            best_model = results
    
    # Fit the VAR model with the best order for the current series
    best_model = VAR(series)
    best_results = best_model.fit(best_order)
    
    # Store the best VAR model for the current series in the list
    best_models.append(best_results)

# Retrieve the VAR parameters for each series
params = np.array([model.params.flatten() for model in best_models])


In [115]:
results = VAR(X[0]).fit(2)

In [116]:
results.aic

LinAlgError: 31-th leading minor of the array is not positive definite

In [97]:
from statsmodels.tsa.api import VAR
params = np.array([VAR(X[i]).fit(2).params.flatten() for i in range(X.shape[0])])

In [120]:
params.shape

(250, 4836)

In [98]:
np.unique(y,return_counts=True)

(array([0, 1]), array([ 81, 169]))

In [121]:
from sklearn.model_selection import train_test_split
from pyrcn.echo_state_network import ESNClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

X_train, X_test, y_train, y_test = train_test_split(params, y, test_size=0.2, random_state=42)
print(np.unique(y_train,return_counts=True))
minority = np.unique(y_train,return_counts=True)[1][0]
mayority = np.unique(y_train,return_counts=True)[1][1]

class_weight = {
    0: 1.0,  
    1: mayority/minority
}

sample_weights = np.array([class_weight[label] for label in y_train])

clf = RandomForestClassifier()

# clf = CatBoostClassifier(task_type='GPU')

clf.fit(X_train, y_train,
        sample_weight=sample_weights
        )


(array([0, 1]), array([ 64, 136]))


RandomForestClassifier()

In [122]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(np.unique(y_test, return_counts=True))
print(np.unique(y_pred, return_counts=True))

              precision    recall  f1-score   support

           0       0.29      0.12      0.17        17
           1       0.65      0.85      0.74        33

    accuracy                           0.60        50
   macro avg       0.47      0.48      0.45        50
weighted avg       0.53      0.60      0.54        50

0.6
(array([0, 1]), array([17, 33]))
(array([0, 1]), array([ 7, 43]))


In [ ]:
arima end

In [18]:
from sklearn.svm import SVC
# from pyrcn.echo_state_network import ESNClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC

def train_3(X, y, verbose=False, display_roc=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    clf = CatBoostClassifier(task_type = 'GPU')
    clf = RandomForestClassifier()
    clf = SVC()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)    
    if verbose:
        print(classification_report(y_test, y_pred))
        print(np.unique(y_test, return_counts=True))
        print(np.unique(y_pred, return_counts=True))
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    if display_roc:
        from sklearn.metrics import roc_curve
        from sklearn.metrics import RocCurveDisplay
        fpr, tpr, _ = roc_curve(y_test, y_pred, pos_label=clf.classes_[1])
        roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
        return roc_display
    return accuracy, f1, roc

In [19]:
numT = X.shape[2]
timepoint_accuracy = [None]*numT
timepoint_f1 = [None]*numT
timepoint_roc = [None]*numT
for timepoint in range(numT):
    print(timepoint)
    X_training = X[:,:,timepoint]
    timepoint_accuracy[timepoint], timepoint_f1[timepoint], timepoint_roc[timepoint] = train_3(X_training, y)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


KeyboardInterrupt: 

In [ ]:
print('f1 mean: ', np.mean(timepoint_f1))
print('acc mean: ', np.mean(timepoint_accuracy))
plt.plot(timepoint_f1)

# RESULTS

In [ ]:
X_training = X[:,:,1500]
print(train_3(X_training, y, verbose=True))

In [ ]:
from sklearn.svm import SVC
# from pyrcn.echo_state_network import ESNClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# scaler = StandardScaler()

clf = CatBoostClassifier(task_type = 'GPU')
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test set:", accuracy)



catboost and RF both on more data with stacked are perfoming the same

next im gonna add labels for the subjects


In [ ]:
display(all_rawdata[0]['metadata'].columns)

In [ ]:
print(multivariate)

# multivariate test?

https://pypi.org/project/pyts/

In [ ]:
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

from pyts.datasets import load_basic_motions

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# X_train, X_test, y_train, y_test = load_basic_motions(return_X_y=True)

print(X_train.shape, y_train.shape)

# scaler = StandardScaler()

clf = CatBoostClassifier(task_type = 'GPU')
clf = RandomForestClassifier()
clf = MultivariateClassifier(BOSSVS())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test set:", accuracy)

